In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

import keras
import tensorflow as tf
from keras import backend as K 
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from tensorflow.python.keras.engine.base_layer import Layer

Using TensorFlow backend.


## Define activation functions

In [4]:
def ReLU(X):
    return tf.maximum(X, np.zeros_like(X))

def softmax(X):
    return tf.exp(X)/tf.reduce_sum(tf.exp(X), 0)

def tanh(X):
    return tf.tanh(X)

## Loading weights from a pre-trained encoder saved in file

In [5]:
weights = np.load('./weights/FinalWeightsArray.npy')
weights1 = weights[0]
weights2 = weights[2]
weights3 = weights[4]
weights4 = weights[6]
weights5 = weights[8]
weights6 = weights[10]
weights7 = weights[12]

print(len(weights))
print(weights1.shape,weights2.shape,weights3.shape, weights4.shape, weights5.shape, weights6.shape, weights7.shape)

14
(3, 3, 1, 16) (3, 3, 16, 8) (3, 3, 8, 8) (3, 3, 8, 8) (3, 3, 8, 8) (3, 3, 8, 16) (3, 3, 16, 1)


## Defining a custom convolution2d function

The function takes tensor as input along with trained kernel weights, kernel size, padding and activation function and outputs a tensor

In [6]:
def customConv2D(input_imgs, weights, filters, kernel, strides=1, padding='same', activation='ReLU'):
    
    if isinstance(kernel, tuple):
        ker_height, ker_weight = kernel
    else:
        ker_height, ker_weight = kernel, kernel
        
    batch , inp_h, inp_w, inp_c = input_imgs.shape
    if padding == 'valid':
        pad = 0
    elif padding == 'same':
        pad = (ker_height - 1)//2
    else:
        raise ValueError('Padding '+padding+' given is incorrect')
    
    conv_height = (inp_h + 2*pad - ker_height)//strides + 1
    conv_weight = (inp_w + 2*pad - ker_weight)//strides + 1
    
    inp_pad = tf.pad(input_imgs, [[0,0],[pad,pad],[pad,pad],[0,0]])
    
    c_list = []
    for i in range(conv_height):
        for j in range(conv_weight):
            c = tf.slice(inp_pad, [0, i*strides, j*strides, 0], [-1, ker_height, ker_weight, -1])
            c_list.append(c)
            
    inp_mat = tf.reshape(tf.stack(c_list), [-1, inp_c * ker_weight * ker_height])
    _, _, ker_c, _ = weights.shape
    W_mat = tf.reshape(weights, [ker_height * ker_weight * ker_c, filters])
    b = tf.zeros([filters])
    
    output = tf.matmul(inp_mat, W_mat) + b
    output = tf.transpose(tf.reshape(output,[conv_height,conv_weight,batch,filters]), [2,0,1,3])
    
    if activation:
        activation = activation.lower()
        if activation == 'relu':
            conv = ReLU(output)
        elif activation == 'softmax' or activation == 'sigmoid':
            conv = softmax(output)
        elif activation == 'tanh':
            conv = tanh(output)
        else:
            raise ValueError('Unknown activation')
        
    return output

## Defining a custom maxpooling 2d function

The function takes tensor as input with kernel size and padding and outputs a max pooled tensor

In [7]:
def customMaxPooling2D(input_imgs, kernel, strides=2, padding='same'):
    
    if isinstance(kernel, tuple):
        ker_height, ker_weight = kernel
    else:
        ker_height, ker_weight = kernel, kernel
        
    _, inp_h, inp_w, inp_c = input_imgs.shape
    
    if padding == 'valid':
        pad = 0
    elif padding == 'same':
        pad = (ker_height-1)//2
    else:
        raise ValueError('Padding '+padding+' given is incorrect')
    
    pool_height = math.ceil(int(inp_h + 2*pad - ker_height)/int(strides)) + 1 
    pool_weight = math.ceil(int(inp_w + 2*pad - ker_weight)/int(strides)) + 1
    
    inp_pad = tf.pad(input_imgs, [[0,0],[pad,pad],[pad,pad],[0,0]])
    
    pool_list = []
    for i in range(pool_height):
        for j in range(pool_weight):
            p = tf.slice(inp_pad, [0, i*strides, j*strides, 0], [-1, ker_height, ker_weight, -1])
            pool_list.append(p)
    
    inp_mat = tf.reshape(tf.stack(pool_list), [-1, ker_weight * ker_height])
    
    pool_output = tf.reduce_max(tf.reshape(inp_mat, [pool_height, pool_weight, -1, ker_height * ker_weight, int(inp_c)]), axis = 3)
    return tf.transpose(pool_output, [2,0,1,3])

## Defining a custom upsampling 2d function

The function takes a tensor, kernel size and data format as input and outputs a upsampled tensor with nearest neighbor interpolation method

In [8]:
def customUpSampling2D(input_imgs, kernel, data_format='channels_last'):
    
    if isinstance(kernel, tuple):
        ker_height, ker_weight = kernel
    else:
        ker_height, ker_weight = kernel, kernel
        
    if (data_format=='channels_last'):
        n, inp_h, inp_w, inp_c = input_imgs.shape
    elif (data_format=='channels_first'):
        n, inp_c, inp_h, inp_w = input_imgs.shape
    else:
        raise ValueError('Data format is not valid.')
    
    out_height = inp_h * ker_height
    out_weight = inp_w * ker_weight
    
    up_list = []
    k = 0
    while k < int(n):
        i = 0
        while i < int(inp_h):
            tmp = []
            j = 0
            while j < int(inp_w):
                p = tf.slice(input_imgs, [k,i,j,0], [1,1,1,1])
                for x in range(ker_weight):
                    tmp.append(p)
                j+=1
            for x in range(ker_height):
                for y in tmp:
                    up_list.append(y)
            i+=1
        k+=1
    inp_mat = tf.reshape(tf.stack(up_list), [n, out_height, out_weight])
    
    out_list = []
    for i in range(int(inp_c)):
        out_list.append(inp_mat)
        
    out_upsamp = tf.reshape(tf.stack(out_list), [n, out_height, out_weight, int(inp_c)])
    
    return out_upsamp

## Load the MNIST dataset

In [9]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1)) 
#channels last approach as grayscale (1) is last parameter
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


## Call the encoder part of the custom network

In [10]:
input_img = x_train[:10,:,:,:]
print(input_img.shape)
print(type(input_img))

x = customConv2D(input_img, weights1, 16, (3,3), activation = 'relu', padding = 'same')
print(x.shape, type(x))
x = customMaxPooling2D(x, (2,2), padding = 'same')
print(x.shape, type(x))
x = customConv2D(x, weights2, 8, (3,3), activation = 'relu', padding = 'same')
print(x.shape, type(x))
x = customMaxPooling2D(x, (2,2), padding = 'same')
print(x.shape, type(x))
x = customConv2D(x, weights3, 8, (3,3), activation = 'relu', padding = 'same')
print(x.shape, type(x))
encoded = customMaxPooling2D(x, (2,2), padding = 'same')

print(encoded.shape, type(x))

(10, 28, 28, 1)
<class 'numpy.ndarray'>
(10, 28, 28, 16) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 14, 14, 16) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 14, 14, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 7, 7, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 7, 7, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 4, 4, 8) <class 'tensorflow.python.framework.ops.Tensor'>


## Call the decoder part of the custom network

In [11]:
x = customConv2D(encoded, weights4, 8, (3,3), activation = 'relu', padding = 'same')
print(x.shape, type(x))
x = customUpSampling2D(x, (2,2))
print(x.shape, type(x))
x = customConv2D(x, weights5, 8, (3,3), activation = 'relu', padding = 'same')
print(x.shape, type(x))
x = customUpSampling2D(x, (2,2))
print(x.shape, type(x))
x = customConv2D(x, weights6, 16, (3,3), activation = 'relu', padding = 'valid')
print(x.shape, type(x))
x = customUpSampling2D(x, (2,2))
print(x.shape, type(x))
decoded = customConv2D(x, weights7, 1, (3,3), activation = 'sigmoid', padding = 'same')

print(decoded.shape, type(x))

(10, 4, 4, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 8, 8, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 8, 8, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 16, 16, 8) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 14, 14, 16) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 28, 28, 16) <class 'tensorflow.python.framework.ops.Tensor'>
(10, 28, 28, 1) <class 'tensorflow.python.framework.ops.Tensor'>


## Plot the decoded images 

In [14]:
n = 10
plt.figure(figsize=(10, 4), dpi=100)

print(decoded.shape)
print(np.ndarray(decoded))
# x = tf.slice(decoded, [0,0,0,0], [1,-1,-1,-1])
# print(x)
# t = tf.Session().run(x)
sess = tf.InteractiveSession()
decoded.eval()

# type(t)

for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.set_axis_off()

    # display reconstruction
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(tf.reshape((decoded[i,:,:,:]), [28, 28]))
    plt.gray()
    ax.set_axis_off()

plt.show()

(10, 28, 28, 1)


TypeError: expected sequence object with len >= 0 or a single integer

<Figure size 1000x400 with 0 Axes>

As the output is a tensor, we needed to convert it into a numpy array before passing it into a matplotlib function to show it as an image. 
The function to do this in tensorflow is 
> sess = tf.InteractiveSession() <br>
> decoded.eval()

But this gives an InvalidArgumentError and there is no source out here which helps with this certain error. Hence, this is where the whole thing is stuck right now.